In [47]:
import geopandas as gpd
import pandas as pd
from pandas import DataFrame
import fiona
from shapely.geometry import shape
# pip install geojson
import geojson
import json
import os
from ipyleaflet import Map, Polygon, DrawControl, basemaps

In [69]:
# Create a map widget and draw bounding box/polygon
m = Map(
    basemap=basemaps.Esri.WorldImagery,
    center=(0,0),
    zoom=1.,
    box_zoom = True
)

draw = DrawControl(circlemarker={}, polyline={})

draw.rectangle = {
    "shapeOptions": {
        "fillColor": "#FFA700",
        "color": "#FFA700",
        "fillOpacity": 0.5
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    feature_collection['features'].append(geo_json)
    print(action)
    print(geo_json)
    return geo_json

draw.on_draw(handle_draw)
m.add_control(draw)

m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [6]:
# Convert feature collection json to variable
bbox = feature_collection['features']
print(bbox)

[{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#FFA700', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': '#FFA700', 'fillOpacity': 0.5, 'clickable': True}}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-78.75, -70.844673], [-78.75, -63.233627], [-52.03125, -63.233627], [-52.03125, -70.844673], [-78.75, -70.844673]]]}}]


In [7]:
# Prints bounds of map object // I would like this to print bounds of bbox
print(m.bounds)

((-80.17871349622823, -200.39062500000003), (80.17871349622823, 201.09375000000003))


In [8]:
# Write bounding box to geojson file
with open('data_bbox2.json', 'w') as f:
    json.dump(bbox, f)

In [9]:
!cat data_bbox2.json

[{"type": "Feature", "properties": {"style": {"stroke": true, "color": "#FFA700", "weight": 4, "opacity": 0.5, "fill": true, "fillColor": "#FFA700", "fillOpacity": 0.5, "clickable": true}}, "geometry": {"type": "Polygon", "coordinates": [[[-78.75, -70.844673], [-78.75, -63.233627], [-52.03125, -63.233627], [-52.03125, -70.844673], [-78.75, -70.844673]]]}}]

In [10]:
shapefile_filepath = str(os.getcwd() + '/data_bbox2.json')
print(shapefile_filepath)

/home/jovyan/icesat2_hackweek/notebooks/data_bbox2.json


In [18]:
#gdf = gpd.read_file(shapefile_filepath)
path = pd.read_json(shapefile_filepath)
print(path)

      type                                         properties  \
0  Feature  {'style': {'stroke': True, 'color': '#FFA700',...   

                                            geometry  
0  {'type': 'Polygon', 'coordinates': [[[-78.75, ...  


In [39]:
gdf = gpd.GeoDataFrame(path)
print(gdf)
# features column does not need to be stored, this is just for illustration
gdf['features'] = gdf['geometry'].apply(lambda x: {'geometry': x, 'properties': {}})
gdf2 = gpd.GeoDataFrame.from_features(gdf['features'])

#gdf = gdf.set_geometry(gdf2.geometry)
gdf2.head()

      type                                         properties  \
0  Feature  {'style': {'stroke': True, 'color': '#FFA700',...   

                                            geometry  \
0  {'type': 'Polygon', 'coordinates': [[[-78.75, ...   

                                            features  
0  {'geometry': {'type': 'Polygon', 'coordinates'...  


,geometry
0,"POLYGON ((-78.75000 -70.84467, -78.75000 -63.2..."


In [40]:
print(gdf2)

                                            geometry
0  POLYGON ((-78.75000 -70.84467, -78.75000 -63.2...


In [44]:
product = gdf2['geometry']
print(product)

0    POLYGON ((-78.75000 -70.84467, -78.75000 -63.2...
Name: geometry, dtype: geometry


In [48]:
df = DataFrame(gdf2, columns= ['geometry'])

In [49]:
df

,geometry
0,"POLYGON ((-78.75000 -70.84467, -78.75000 -63.2..."
